In [ ]:
# Training an FNO for the Darcy flow Problem

In [ ]:
pip install torch

In [1]:
import torch
import numpy as np
import scipy.io
import h5py
import torch.nn as nn
from sklearn.model_selection import train_test_split

import operator
from functools import reduce
from functools import partial

import torch.nn.functional as F

import matplotlib.pyplot as plt

from timeit import default_timer

from torch.optim import Adam

from FNO2D import *
from utilities3 import *

torch.manual_seed(0)
np.random.seed(0)
torch.set_printoptions(precision=8)

In [2]:
# load the data
reader = MatReader('df_train_128.mat')
coeff = reader.read_field('coeff')
sol = reader.read_field('sol')


In [3]:
# split into train/ validation : 80 / 20
x_train, x_val, y_train, y_val = train_test_split(coeff, sol, random_state=0, test_size=0.20)

In [4]:
# hyperparameters # 

ntrain = 800
nval = 200

batch_size = 20
learning_rate = 0.001

epochs = 100
step_size = 20
gamma = 0.5

modes = 12
width = 32


s = 128


In [5]:
# data normalisation
x_normalizer = UnitGaussianNormalizer(x_train)
x_train = x_normalizer.encode(x_train)
x_val = x_normalizer.encode(x_val)

y_normalizer = UnitGaussianNormalizer(y_train)
y_train = y_normalizer.encode(y_train)

x_train = x_train.reshape(ntrain,s,s,1)
x_val = x_val.reshape(nval,s,s,1)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_val, y_val), batch_size=batch_size, shuffle=False)

In [6]:
# training the FNO

model = FNO2d(modes, modes, width) #.cuda()

optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)
t1 = default_timer()

# y_normalizer.cuda()
for ep in range(epochs):
    model.train()
    train_l2 = 0
    
    for x, y in train_loader:
        # x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        out = model(x).reshape(batch_size, s, s)
        out = y_normalizer.decode(out)
        y = y_normalizer.decode(y)

        loss = myloss(out.view(batch_size,-1), y.view(batch_size,-1))
        
        loss.backward()

        optimizer.step()
        train_l2 += loss.item()

    scheduler.step()

    model.eval()
    val_l2 = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            # x, y = x.cuda(), y.cuda()

            out = model(x).reshape(batch_size, s, s)
            out = y_normalizer.decode(out)

            val_l2 += myloss(out.view(batch_size,-1), y.view(batch_size,-1)).item()

    train_l2/= ntrain
    val_l2 /= nval

    t2 = default_timer()
    print("Ep:",ep,"Time:", t2-t1, "Train L:", train_l2, "Val L:", val_l2)

Ep: 0 Time: 362.11250189878047 Train L: 0.09227138303220273 Val L: 0.047715665102005006
Ep: 1 Time: 721.6687843063846 Train L: 0.04679346866905689 Val L: 0.04577301174402237
Ep: 2 Time: 1083.8692088825628 Train L: 0.045174507200717924 Val L: 0.0432310476899147
Ep: 3 Time: 1443.8173845093697 Train L: 0.04027597136795521 Val L: 0.03754954010248184
Ep: 4 Time: 1803.7922482276335 Train L: 0.034624925814568996 Val L: 0.02924569383263588
Ep: 5 Time: 2165.0937876589596 Train L: 0.03086181778460741 Val L: 0.02881988450884819
Ep: 6 Time: 2525.8056990392506 Train L: 0.02948594782501459 Val L: 0.030938892811536788
Ep: 7 Time: 2887.559965084307 Train L: 0.031622606739401815 Val L: 0.02823963239789009
Ep: 8 Time: 3248.08435650263 Train L: 0.028625593818724156 Val L: 0.028022406399250032
Ep: 9 Time: 3608.4364790041 Train L: 0.02824628561735153 Val L: 0.027430027723312378
Ep: 10 Time: 3970.9337970241904 Train L: 0.029693724848330022 Val L: 0.027395018190145493
Ep: 11 Time: 4330.832236493938 Train L: 

In [7]:

# Load data to test the model

reader = MatReader('df_test_256.mat')
x_test_256 = reader.read_field('coeff')
x_normalizer_256 = UnitGaussianNormalizer(x_test_256)
y_test_256 = reader.read_field('sol')
y_normalizer_256 = UnitGaussianNormalizer(y_test_256)
x_test_256 = x_normalizer_256.encode(x_test_256)
x_test_256 = x_test_256.reshape(200,256,256,1)



reader.load_file('df_test_128.mat')
x_test_128 = reader.read_field('coeff')
x_normalizer_128 = UnitGaussianNormalizer(x_test_128)
y_test_128 = reader.read_field('sol')
y_normalizer_128 = UnitGaussianNormalizer(y_test_128)
x_test_128 = x_normalizer_128.encode(x_test_128)
x_test_128 = x_test_128.reshape(200,128,128,1)



reader.load_file('df_test_64.mat')
x_test_64 = reader.read_field('coeff')
x_normalizer_64 = UnitGaussianNormalizer(x_test_64)
y_test_64 = reader.read_field('sol')
y_normalizer_64 = UnitGaussianNormalizer(y_test_64)
x_test_64 = x_normalizer_64.encode(x_test_64)
x_test_64 = x_test_64.reshape(200,64,64,1)

test_loader_256 = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test_256, y_test_256), batch_size=1, shuffle=False)

test_loader_128 = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test_128, y_test_128), batch_size=1, shuffle=False)

test_loader_64 = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test_64, y_test_64), batch_size=1, shuffle=False)



In [8]:
# Testing the FNO on 256 resolution

mytestloss = LpLoss(size_average=False)
t1 = default_timer()

model.eval()
test_l2 = 0.0
with torch.no_grad():
    for x, y in test_loader_256:
                # x, y = x.cuda(), y.cuda()
            
        out = model(x).reshape(1, 256, 256)
        out = y_normalizer_256.decode(out)

        test_l2 += mytestloss(out.view(1,-1), y.view(1,-1)).item()


test_l2 /= 200

t2 = default_timer()
time_taken_256 = t2-t1
loss_256 = test_l2
print("resolution: 256", "Time taken:", t2-t1, "Testing Loss:", test_l2)

resolution: 256 Time taken: 126.03673484455794 Testing Loss: 0.034208891238085924


In [9]:
# Testing the FNO on 128 resolution

mytestloss = LpLoss(size_average=False)
t1 = default_timer()

model.eval()
test_l2 = 0.0
with torch.no_grad():
    for x, y in test_loader_128:
                # x, y = x.cuda(), y.cuda()
            
        out = model(x).reshape(1, 128, 128)
        out = y_normalizer_128.decode(out)

        test_l2 += mytestloss(out.view(1,-1), y.view(1,-1)).item()


test_l2 /= 200

t2 = default_timer()
time_taken_128 = t2-t1
loss_128 = test_l2
print("resolution: 128", "Time taken:", t2-t1, "Testing Loss:", test_l2)

resolution: 128 Time taken: 30.21675187535584 Testing Loss: 0.03279689413728192


In [10]:
# Testing the FNO on 64 resolution

mytestloss = LpLoss(size_average=False)
t1 = default_timer()

model.eval()
test_l2 = 0.0
with torch.no_grad():
    for x, y in test_loader_64:
                # x, y = x.cuda(), y.cuda()
            
        out = model(x).reshape(1, 64, 64)
        out = y_normalizer_64.decode(out)

        test_l2 += mytestloss(out.view(1,-1), y.view(1,-1)).item()


test_l2 /= 200

t2 = default_timer()
time_taken_64 = t2-t1
loss_64 = test_l2
print("resolution: 64", "Time taken:", t2-t1, "Testing Loss:", test_l2)

resolution: 64 Time taken: 10.518008392304182 Testing Loss: 0.03542569748591631


In [11]:
# Save the results to compare to FDM
resolution = [256, 128, 64]
time_taken = [time_taken_256, time_taken_128, time_taken_64]
testing_loss = [loss_256, loss_128, loss_64]
scipy.io.savemat('df_FNO_results.mat', mdict={'res': resolution, 'time':time_taken, 'loss': testing_loss})